In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA
from tqdm import tqdm
import pickle
import faiss
from transformers import AutoTokenizer, AutoModel

# Set up MLP Model

In [1]:
class BERTEmbeddingTransform(object):
    def __init__(self, bert_model, tokenizer, device='cpu'):
        bert_model.eval()
        bert_model = bert_model.to(device)
        bert_model.share_memory()
        self.bert_model = bert_model
        self.tokenizer = tokenizer
        self.device = device
    
    def __call__(self, sample):
        code_tokens=self.tokenizer.tokenize(sample)
        tokens = code_tokens
        tokens_ids=self.tokenizer.convert_tokens_to_ids(tokens)
        done_tok = torch.split(torch.tensor(tokens_ids, device=self.device), 510)
        with torch.no_grad():
            embedings = []
            for input_tok in done_tok:
                input_tok = torch.cat((torch.tensor([0], device=self.device), input_tok, torch.tensor([2], device=self.device)))
                temp = self.bert_model(input_tok.clone().detach()[None,:], output_hidden_states = True)
                embedings.append(temp[1][-2])
            return torch.concat(embedings,dim=1).squeeze().mean(dim=0)

In [5]:
class MLP256(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(768, 512),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(512, 512),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(512, 512),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(512, 256),
        )
    def forward(self, x):
        y = self.mlp(x)
        return y

## Set up BERT

In [6]:
tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-cpp")
BERT = AutoModel.from_pretrained("neulab/codebert-cpp", add_pooling_layer = False)
BERT.eval()
if torch.cuda.is_available():
    bert_transform = BERTEmbeddingTransform(BERT,tokenizer, 'cuda')
else:
    bert_transform = BERTEmbeddingTransform(BERT,tokenizer, 'cpu')

Some weights of the model checkpoint at neulab/codebert-cpp were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Set up MLP

In [7]:
mlp = MLP256()
#model = BiLSTMVectorizer(768, 256)
if torch.cuda.is_available():
    mlp.to("cuda")
mlp.load_state_dict(torch.load("../models/MLP256_last.pth", map_location=torch.device('cpu')))
mlp.eval()

MLP256(
  (mlp): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=512, out_features=256, bias=True)
  )
)

# Load Data

In [11]:
faiss_index = faiss.read_index('../data/external/task_index_median_MLP256_03-04-23.bin')
keys_df = pd.read_csv("../data/external/keys_df.csv", index_col=0)
pwt_df = pd.read_csv("../data/external/codeforces-problems.csv", index_col=0)

In [12]:
keys_df.head()

,problem_url
0,/contest/1/problem/A
1,/contest/1/problem/B
2,/contest/1/problem/C
3,/contest/2/problem/A
4,/contest/2/problem/B


In [13]:
pwt_df["problem_url"] = pwt_df["problem_url"].apply(lambda x: x.replace("contests", "contest"))
pwt_df['problem_tags'] = pwt_df['problem_tags'].astype(str)
pwt_df.head()

,contest,problem_name,problem_statement,problem_tags,rating,problem_url
0,325,A,You are given n rectangles. The corners of rec...,implementation,1500.0,/contest/325/problem/A
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math",1800.0,/contest/325/problem/B
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths",2600.0,/contest/325/problem/C
3,325,D,"In a far away land, there exists a planet shap...",dsu,2900.0,/contest/325/problem/D
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy",2800.0,/contest/325/problem/E


In [14]:
merged_df = pd.merge(keys_df, pwt_df, how="left", on='problem_url')
merged_df.head()

,problem_url,contest,problem_name,problem_statement,problem_tags,rating
0,/contest/1/problem/A,1.0,A,Theatre Square in the capital city of Berland ...,math,1000.0
1,/contest/1/problem/B,1.0,B,In the popular spreadsheets systems (for examp...,"implementation,math",1600.0
2,/contest/1/problem/C,1.0,C,Nowadays all circuses in Berland have a round ...,"geometry,math",2100.0
3,/contest/2/problem/A,2.0,A,The winner of the card game popular in Berland...,"hashing,implementation",1500.0
4,/contest/2/problem/B,2.0,B,"There is a square matrix n × n, consisting of ...","dp,math",2000.0


# Test FAISS

In [15]:
source_code = '#include <bits/stdc++.h>\n \nusing namespace std;\ntypedef long long ll;\n \nvector<vector<int>> adj,btree;\nint timer=1,bid=1;\nvector<int> vis,tin,low;\nset<pair<int,int>> edges;\n \nvoid bridge(int v,int p){\n    vis[v]=1;\n    tin[v] = low[v] = timer++;\n    for(auto u : adj[v]){\n        if(u == p) continue;\n        if(vis[u]){\n            low[v] = min(low[v],tin[u]);\n        }else{\n            bridge(u,v);\n            low[v] = min(low[v],low[u]);\n            if(low[u] > tin[v]){\n                // bridge   \n                edges.insert({min(u,v),max(u,v)});\n            }\n        }\n    }\n}\n \n \nvoid build(int v,int cur){\n    vis[v]=1;\n    for(auto u : adj[v]){\n        if(vis[u])continue;\n        if(edges.find({min(u,v),max(u,v)}) == edges.end()){\n            build(u,cur);\n        }else{\n            bid++;\n            btree[cur].push_back(bid);\n            btree[bid].push_back(cur);\n            build(u,bid);\n        }\n    }\n}\n \nint ans =0,s;\nvoid dfs(int v,int p,int cur){\n    cur++;\n    if(cur > ans){\n        ans = cur,s =v;\n    }\n    for(auto u : btree[v]){\n        if(u == p)continue;\n        dfs(u,v,cur);\n    }\n}\n \nvoid solve(){\n    int n,m;cin>>n>>m;\n    adj.resize(n+1),btree.resize(n+1);\n    for(int i=0;i<m;i++){\n        int x,y;cin>>x>>y;\n        adj[x].push_back(y);\n        adj[y].push_back(x);\n    }\n    vis.resize(n+1,0),tin.resize(n+1,0),low.resize(n+1,1e9);\n    bridge(1,-1);\n    vis.clear(),vis.resize(n+1,0);\n    build(1,1);\n    dfs(1,-1,0);\n    dfs(s, -1,0);\n    cout<<ans-1;\n}\n \n \nint main(){\n   ios_base::sync_with_stdio(0),cin.tie(0);\n   int t=1;//cin>>t;\n   while(t--) solve();\n}\n'

In [16]:
emb = mlp(bert_transform(source_code))
emb.size()

Token indices sequence length is longer than the specified maximum sequence length for this model (968 > 512). Running this sequence through the model will result in indexing errors


torch.Size([256])

In [17]:
result = faiss_index.search(emb.detach().numpy().reshape(-1,256), k=20)
result

(array([[243513.67, 261347.53, 311992.  , 321059.  , 330046.12, 357358.03,
         360204.75, 360420.5 , 362626.97, 368758.06, 370713.16, 386262.06,
         389857.4 , 390099.53, 402195.38, 409448.38, 409600.75, 412134.3 ,
         436164.  , 438096.03]], dtype=float32),
 array([[3071, 2413, 5716, 2013, 1062, 1069, 1605, 1920, 5093, 5136, 5255,
         4606,   61, 3404, 2256, 6950, 2258,  819, 6710, 2020]]))

In [18]:
for i in result[1][0]:
    print(merged_df.problem_url[i], merged_df.problem_tags[i])

/contest/687/problem/E dfsandsimilar,graphs
/contest/542/problem/E graphs,shortestpaths
/contest/1220/problem/E dfsandsimilar,dp,dsu,graphs,greedy,trees
/contest/449/problem/B graphs,greedy,shortestpaths
/contest/229/problem/B binarysearch,datastructures,graphs,shortestpaths
/contest/230/problem/D binarysearch,graphs,shortestpaths
/contest/346/problem/D dp,graphs,shortestpaths
/contest/427/problem/C dfsandsimilar,graphs,twopointers
/contest/1101/problem/D datastructures,dfsandsimilar,dp,numbertheory,trees
/contest/1108/problem/F binarysearch,dsu,graphs,greedy
/contest/1139/problem/E flows,graphmatchings,graphs
/contest/1000/problem/E dfsandsimilar,graphs,trees
/contest/14/problem/D dfsandsimilar,dp,graphs,shortestpaths,trees,twopointers
/contest/757/problem/F datastructures,graphs,shortestpaths
/contest/505/problem/D dfsandsimilar
/contest/1467/problem/E nan
/contest/506/problem/B dfsandsimilar,graphs
/contest/178/problem/B1 nan
/contest/1423/problem/B nan
/contest/450/problem/D graphs

## Test once again

In [20]:
my_code = """#include <bits/stdc++.h>
#define ll long  long
#define f first
#define s second
#define pii pair<int,int>
#define pll pair<ll,ll>
#define pb push_back
#define epb emplace_back
using namespace std;

const int nmax = 500001;
char ask(vector <int> vec){
    cout << "? ";
    cout << vec.size() << "\n";
    for(int to : vec)
        cout << to << " ";
    cout << "\n";
    string s; cin >> s;
    return s[0];
}

vector <int> add(vector <int> a, vector <int> b){
    for(int to : b)
        a.pb(to);
    return a;
}

int main(){
  // ios_base::sync_with_stdio(false); cin.tie(0);
    int n; cin >> n;
    vector <int> vec;
    for(int i = 1; i <= n; i++){
        vec.pb(i);
    }
    while(vec.size() > 3){
        vector <int> v[4];
        for(int i = 0; i < vec.size(); i++){
            v[i % 4].pb(vec[i]);
        }
        char c = ask(add(v[0], v[1]));
        char d = ask(add(v[0], v[2]));
        if(c == 'Y' && d == 'Y'){
            vec = add(add(v[0], v[1]), v[2]);
        }
        if(c == 'N' && d == 'N'){
            vec = add(add(v[1], v[2]), v[3]);
        }
        if(c == 'Y' && d == 'N'){
            vec = add(add(v[0], v[1]), v[3]);
        }
        if(d == 'Y' && c == 'N'){
            vec = add(add(v[0], v[2]), v[3]);
        }
    }
    while(vec.size() > 2){
        vector <int> v[3];
        for(int i = 0; i < 3; i++){
            v[i].pb(vec[i]);
        }
        char c[4];
        c[0] = ask(v[0]);
        c[1] = ask(v[1]);
        c[2] = ask(v[1]);
        c[3] = ask(v[0]);
        if(c[1] == c[2] && c[1] == 'N'){
            vec = add(v[0], v[2]);
            break;
        }
        bool ind = false;
        for(int i = 0; i < 3; i++){
            if(c[i] == c[i + 1] && c[i] == 'Y')
                ind = true;
        }
        if(ind)
            vec = add(v[0], v[1]);
        else vec = add(v[1], v[2]);
    }
    for(int i = 0; i < vec.size(); i++){
        cout << "! " << vec[i] << "\n";
        string s; cin >> s;
        if(s == ":)"){
            return 0;
        }
    }
    return 0;
}

/* - - - - - - - - - - - - - -
  |             ##            |
  |      #      ##      #     |
  |    #####    ##    #####   |
  |      #      ##      #     |
  |             ##            |
  | ##########################|
  |             ##            |
  |      #      ##      #     |
  |    #####    ##    #####   |
  |      #      ##      #     |
  |             ##            |
   - - - - - - - - - - - - - -
*/
"""

In [21]:
my_emb = mlp(bert_transform(my_code))

my_emb.size()

torch.Size([256])

In [22]:
my_recs = faiss_index.search(my_emb.detach().numpy().reshape(-1,256), k=40)
my_recs

(array([[305015.47, 469340.  , 475558.9 , 485043.06, 495876.1 , 500925.38,
         518346.38, 520242.94, 551583.6 , 555028.06, 558868.7 , 566499.6 ,
         576634.94, 580273.06, 592506.  , 612349.75, 625550.5 , 627672.8 ,
         635495.4 , 645210.44, 659217.4 , 676676.94, 678242.9 , 692588.56,
         694291.56, 709770.6 , 711381.8 , 733038.6 , 733685.  , 740591.1 ,
         746606.44, 751325.6 , 753392.8 , 754402.6 , 760106.7 , 760855.94,
         761555.6 , 767999.94, 773481.4 , 774505.25]], dtype=float32),
 array([[6127, 6237, 3695, 3912, 7446, 3070, 2044,  753, 7208, 2322, 6811,
         2051, 7618, 6817,  862, 2098, 3788, 5410, 7479, 7738, 1838,  868,
         7415, 2512, 5993,  819, 3702, 6099, 3166, 7315,  820, 1356, 7928,
         1595, 1783, 7803, 6062, 4858,  285, 5986]]))

In [23]:
for i in my_recs[1][0]:
    print(merged_df.problem_url[i], merged_df.problem_tags[i])

/contest/1304/problem/E datastructures,dfsandsimilar,shortestpaths,trees
/contest/1328/problem/E dfsandsimilar,graphs,trees
/contest/812/problem/D dfsandsimilar,graphs,implementation,trees
/contest/855/problem/D trees
/contest/1558/problem/E nan
/contest/687/problem/D bruteforce,datastructures,dsu,graphs,sortings
/contest/455/problem/C dfsandsimilar,dp,dsu,ternarysearch,trees
/contest/165/problem/D datastructures,dsu,trees
/contest/1515/problem/G nan
/contest/519/problem/E binarysearch,datastructures,dfsandsimilar,dp,trees
/contest/1441/problem/D nan
/contest/456/problem/E dfsandsimilar,dsu,graphs,trees
/contest/1594/problem/D nan
/contest/1442/problem/E binarysearch,constructivealgorithms,dfsandsimilar,dp,greedy,trees
/contest/187/problem/C dfsandsimilar,dsu
/contest/466/problem/E dfsandsimilar,dsu,graphs,trees
/contest/832/problem/D dfsandsimilar,graphs,trees
/contest/1166/problem/F datastructures,dsu,graphs,hashing
/contest/1563/problem/E nan
/contest/1615/problem/D nan
/contest/406